In [15]:
!rm -f minsearch.py
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-08-20 12:01:40--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-08-20 12:01:40 (45.7 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [16]:
import requests 
import minsearch

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [17]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [18]:
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [20]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt


def llm(prompt):
    response = client.chat.completions.create(
        model='phi3',
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

In [22]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [24]:
llm('write that this is a test')

'This document serves as a placeholder or proof of concept to demonstrate functionality. It contains the following text segment: "The purpose of this sample input/output (I/O) file, designated for testing software interactions and output validity, may include but isn\'t limited to verifying that all components properly handle expected inputs across various use-case scenarios."\n\nIn accordance with your instruction, here is the test text: "This is a test."'

In [25]:
rag("Is it too late to join the course?")

"As per the given context from FAQ: Yes, even after the course starts on January 15th, it is possible to join and submit homeworks later; however, there are deadlines for turning in final projects, so planning accordingly would be beneficial to avoid leaving everything until the last minute. Late submissions of assignments will not be allowed post due dates but under certain conditions involving unclosed forms as mentioned - details regarding this were provided on an older news source which implies that specific guidance or updates might have been issued previously concerning late submission circumstances and potential closeness after course end, which should also align with the policy for capstone project submissions. As such, it is recommended to submit all work in a timely manner based on official guidelines; if uncertain about any aspect regarding extensions post due dates as stated or needed beforehand beyond explicit permissions that can't be provided here due to being outdated i